# Module 27: LangGraph

**Building Stateful Multi-Agent Applications**

---

## 1. Objectives

- ✅ Understand graph-based agent architecture
- ✅ Build stateful workflows with nodes and edges
- ✅ Implement conditional routing
- ✅ Create multi-agent systems
- ✅ Add human-in-the-loop workflows

## 2. Prerequisites

- [Module 26: LangChain](../26_langchain/26_langchain.ipynb)

## 3. Why LangGraph?

### Limitations of Basic Agents

| Problem | LangGraph Solution |
|---------|-------------------|
| Linear flow only | Graph structure |
| No state persistence | Built-in state management |
| Hard to debug | Visual graph representation |
| No human intervention | Human-in-the-loop nodes |

### LangGraph Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                      LangGraph Workflow                      │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│     ┌─────────┐         ┌─────────┐         ┌─────────┐     │
│     │  START  │ ──────→ │ Node A  │ ──────→ │ Node B  │     │
│     └─────────┘         └─────────┘         └────┬────┘     │
│                                                   │          │
│                              ┌────────────────────┴──────┐   │
│                              ▼                           ▼   │
│                        ┌─────────┐                ┌─────────┐│
│                        │ Node C  │                │  END    ││
│                        └────┬────┘                └─────────┘│
│                              │                               │
│                              └───────────────────────────────│
│                                                              │
│     State: {messages: [...], data: {...}}                    │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Core Concepts

| Concept | Description |
|---------|-------------|
| **State** | Shared data passed between nodes |
| **Nodes** | Functions that transform state |
| **Edges** | Connections between nodes |
| **Conditional Edges** | Dynamic routing based on state |

In [ ]:
# Install: pip install langgraph langchain-openai

from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import operator

print("LangGraph imported!")

## 4. Defining State

### Theory

State is a **TypedDict** that flows through the graph:
- Each node can read and update state
- State updates are **merged** (using reducers)
- Enables persistence and checkpointing

In [ ]:
from typing import TypedDict, Annotated, List
import operator

# Define state schema
class AgentState(TypedDict):
    # Messages accumulate using the add operator
    messages: Annotated[List[BaseMessage], operator.add]
    # Current step for tracking
    current_step: str
    # Any additional data
    data: dict

# Initial state
initial_state: AgentState = {
    "messages": [],
    "current_step": "start",
    "data": {}
}

print("State schema defined!")
print(f"State keys: {list(AgentState.__annotations__.keys())}")

## 5. Building a Simple Graph

In [ ]:
from langgraph.graph import StateGraph, END

# Simple state
class SimpleState(TypedDict):
    value: int
    history: Annotated[List[str], operator.add]

# Define nodes (functions that transform state)
def add_one(state: SimpleState) -> dict:
    """Add 1 to value."""
    return {
        "value": state["value"] + 1,
        "history": [f"Added 1, now {state['value'] + 1}"]
    }

def multiply_two(state: SimpleState) -> dict:
    """Multiply value by 2."""
    return {
        "value": state["value"] * 2,
        "history": [f"Multiplied by 2, now {state['value'] * 2}"]
    }

# Build the graph
workflow = StateGraph(SimpleState)

# Add nodes
workflow.add_node("add", add_one)
workflow.add_node("multiply", multiply_two)

# Add edges
workflow.set_entry_point("add")
workflow.add_edge("add", "multiply")
workflow.add_edge("multiply", END)

# Compile
app = workflow.compile()

# Run
result = app.invoke({"value": 5, "history": []})
print(f"Final value: {result['value']}")
print(f"History: {result['history']}")

In [ ]:
# Visualize the graph
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    # Fallback: print mermaid diagram
    print(app.get_graph().draw_mermaid())

## 6. Conditional Routing

### Theory

Use conditional edges to create **dynamic workflows**:

```python
def router(state) -> str:
    if condition:
        return "node_a"
    else:
        return "node_b"
```

In [ ]:
class RouterState(TypedDict):
    query: str
    category: str
    response: str

def classify_query(state: RouterState) -> dict:
    """Classify the query type."""
    query = state["query"].lower()
    
    if "code" in query or "python" in query:
        category = "technical"
    elif "weather" in query or "time" in query:
        category = "factual"
    else:
        category = "general"
    
    return {"category": category}

def handle_technical(state: RouterState) -> dict:
    return {"response": "🔧 Technical response: Let me help with your code question."}

def handle_factual(state: RouterState) -> dict:
    return {"response": "📊 Factual response: Here are the facts you requested."}

def handle_general(state: RouterState) -> dict:
    return {"response": "💬 General response: I'd be happy to help with that."}

# Router function
def route_query(state: RouterState) -> str:
    """Route to appropriate handler based on category."""
    return state["category"]

# Build graph with conditional routing
router_graph = StateGraph(RouterState)

router_graph.add_node("classify", classify_query)
router_graph.add_node("technical", handle_technical)
router_graph.add_node("factual", handle_factual)
router_graph.add_node("general", handle_general)

router_graph.set_entry_point("classify")

# Conditional edge based on category
router_graph.add_conditional_edges(
    "classify",
    route_query,
    {
        "technical": "technical",
        "factual": "factual",
        "general": "general"
    }
)

router_graph.add_edge("technical", END)
router_graph.add_edge("factual", END)
router_graph.add_edge("general", END)

router_app = router_graph.compile()

# Test different queries
queries = [
    "How do I write Python code?",
    "What's the weather like?",
    "Tell me a story"
]

for q in queries:
    result = router_app.invoke({"query": q, "category": "", "response": ""})
    print(f"Query: '{q}'")
    print(f"  Category: {result['category']}")
    print(f"  Response: {result['response']}\n")

## 7. ReAct Agent Pattern

### Theory

ReAct (Reasoning + Acting) loops:
1. **Think**: Reason about what to do
2. **Act**: Execute a tool
3. **Observe**: Get result
4. **Repeat** until done

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# Define tools
@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Search results for '{query}': Found relevant information."

@tool
def calculator(expression: str) -> str:
    """Calculate a math expression."""
    try:
        return str(eval(expression))
    except:
        return "Error"

tools = [search, calculator]

# LLM with tools
llm = ChatOpenAI(model="gpt-3.5-turbo").bind_tools(tools)

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

class MessagesState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

def call_model(state: MessagesState) -> dict:
    """Call the LLM."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# Create tool node
tool_node = ToolNode(tools)

# Build ReAct graph
react_graph = StateGraph(MessagesState)

react_graph.add_node("agent", call_model)
react_graph.add_node("tools", tool_node)

react_graph.set_entry_point("agent")

# Conditional: if tool calls exist, go to tools; else end
react_graph.add_conditional_edges(
    "agent",
    tools_condition,  # Built-in condition for tool calls
)

# After tools, go back to agent
react_graph.add_edge("tools", "agent")

react_agent = react_graph.compile()

# Run
result = react_agent.invoke({
    "messages": [HumanMessage(content="What is 25 * 4?")]
})

print("Conversation:")
for msg in result["messages"]:
    print(f"  {msg.type}: {msg.content[:100]}..." if len(str(msg.content)) > 100 else f"  {msg.type}: {msg.content}")

## 8. Multi-Agent Systems

### Theory

Multiple specialized agents collaborate:

```
┌─────────────────────────────────────────┐
│            Supervisor Agent              │
│                   │                      │
│     ┌─────────────┼─────────────┐       │
│     ▼             ▼             ▼       │
│ [Researcher] [Writer]    [Reviewer]     │
│     │             │             │       │
│     └─────────────┴─────────────┘       │
│                   │                      │
│              Final Output                │
└─────────────────────────────────────────┘
```

In [ ]:
class MultiAgentState(TypedDict):
    task: str
    research: str
    draft: str
    review: str
    final: str

def researcher(state: MultiAgentState) -> dict:
    """Research agent gathers information."""
    task = state["task"]
    # In real implementation, this would call an LLM
    research = f"📚 Research findings for '{task}': Found 3 key points..."
    return {"research": research}

def writer(state: MultiAgentState) -> dict:
    """Writer agent creates draft."""
    research = state["research"]
    draft = f"📝 Draft based on research: Here's a well-structured article..."
    return {"draft": draft}

def reviewer(state: MultiAgentState) -> dict:
    """Reviewer agent provides feedback."""
    draft = state["draft"]
    review = f"✅ Review: The draft is good with minor suggestions..."
    return {"review": review}

def finalizer(state: MultiAgentState) -> dict:
    """Combine everything into final output."""
    final = f"🎯 Final: Task '{state['task']}' completed successfully!"
    return {"final": final}

# Build multi-agent workflow
multi_agent = StateGraph(MultiAgentState)

multi_agent.add_node("researcher", researcher)
multi_agent.add_node("writer", writer)
multi_agent.add_node("reviewer", reviewer)
multi_agent.add_node("finalizer", finalizer)

multi_agent.set_entry_point("researcher")
multi_agent.add_edge("researcher", "writer")
multi_agent.add_edge("writer", "reviewer")
multi_agent.add_edge("reviewer", "finalizer")
multi_agent.add_edge("finalizer", END)

multi_app = multi_agent.compile()

# Run
result = multi_app.invoke({
    "task": "Write about AI in healthcare",
    "research": "", "draft": "", "review": "", "final": ""
})

print("Multi-Agent Workflow Results:")
for key in ["research", "draft", "review", "final"]:
    print(f"  {key}: {result[key]}")

## 9. Human-in-the-Loop

### Theory

Pause workflow for human approval or input:
- **Checkpoints**: Save state for later resumption
- **Interrupts**: Pause at specific nodes
- **Approval gates**: Require human confirmation

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

class ApprovalState(TypedDict):
    task: str
    plan: str
    approved: bool
    result: str

def create_plan(state: ApprovalState) -> dict:
    return {"plan": f"Plan for '{state['task']}': Step 1, Step 2, Step 3"}

def execute_plan(state: ApprovalState) -> dict:
    return {"result": f"✅ Executed plan: {state['plan']}"}

def check_approval(state: ApprovalState) -> str:
    """Route based on approval status."""
    if state.get("approved", False):
        return "execute"
    return "wait"  # Would pause here in real implementation

# Build with checkpointing
memory = MemorySaver()

approval_graph = StateGraph(ApprovalState)
approval_graph.add_node("plan", create_plan)
approval_graph.add_node("execute", execute_plan)

approval_graph.set_entry_point("plan")
approval_graph.add_edge("plan", "execute")  # Simplified for demo
approval_graph.add_edge("execute", END)

# Compile with checkpointing
approval_app = approval_graph.compile(checkpointer=memory)

# Run with thread ID for persistence
config = {"configurable": {"thread_id": "user-123"}}

result = approval_app.invoke(
    {"task": "Deploy to production", "plan": "", "approved": True, "result": ""},
    config
)

print(f"Result: {result['result']}")

## 10. Persistence and State Management

In [ ]:
# Get conversation history from checkpointer
state_snapshot = approval_app.get_state(config)

print("Current state:")
print(f"  Values: {state_snapshot.values}")
print(f"  Next nodes: {state_snapshot.next}")

# List all checkpoints
print("\nState history available for debugging and replay")

## 11. Interview Questions

**Q1: When should you use LangGraph vs LangChain agents?**
<details><summary>Answer</summary>

Use LangGraph when you need:
- Explicit control over agent flow
- State persistence across sessions
- Human-in-the-loop workflows
- Multi-agent coordination
- Complex conditional routing

Use LangChain agents for simpler, single-agent tasks.
</details>

**Q2: How does state management work in LangGraph?**
<details><summary>Answer</summary>

- State is a TypedDict passed between nodes
- Nodes return partial state updates (merged automatically)
- Reducers (like `operator.add`) define how updates merge
- Checkpointers persist state across invocations
</details>

**Q3: How do you implement human-in-the-loop?**
<details><summary>Answer</summary>

1. Use `interrupt_before` or `interrupt_after` on nodes
2. Save state with a checkpointer (MemorySaver, PostgresSaver)
3. Resume with `app.invoke(None, config)` after human input
4. Update state with `app.update_state(config, new_values)`
</details>

## 12. Summary

| Concept | Purpose |
|---------|--------|
| StateGraph | Define workflow as nodes + edges |
| Conditional Edges | Dynamic routing |
| Checkpointers | Persist state |
| Multi-Agent | Specialized agents collaborate |
| Human-in-the-loop | Pause for human input |

## 13. References

- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [LangGraph Examples](https://github.com/langchain-ai/langgraph/tree/main/examples)
- [Multi-Agent Tutorial](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)

---
**Next:** [Module 28: LlamaIndex](../28_llamaindex/28_llamaindex.ipynb)